In [1]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt # plotting
import numpy as np
import pandas as pd
from IPython.display import display

In [2]:
dataset_17 = pd.read_csv("./Dataset/2017.csv")
dataset_18 = pd.read_csv("./Dataset/2018.csv")

dataset = dataset_17.append(dataset_18, ignore_index=True)


In [3]:
print("shape :",dataset.shape[0],dataset.shape[1])

shape : 12888067 28


In [4]:

display(dataset.head(n=10))

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2017-01-01,AA,1,JFK,LAX,800,831.0,31.0,25.0,856.0,...,402.0,398.0,347.0,2475.0,27.0,0.0,0.0,0.0,0.0,NaN
1,2017-01-01,AA,2,LAX,JFK,900,934.0,34.0,34.0,1008.0,...,327.0,335.0,289.0,2475.0,34.0,0.0,8.0,0.0,0.0,NaN
2,2017-01-01,AA,4,LAX,JFK,1130,1221.0,51.0,20.0,1241.0,...,328.0,319.0,284.0,2475.0,7.0,0.0,0.0,0.0,35.0,NaN
3,2017-01-01,AA,5,DFW,HNL,1135,1252.0,77.0,19.0,1311.0,...,517.0,537.0,513.0,3784.0,77.0,0.0,20.0,0.0,0.0,NaN
4,2017-01-01,AA,6,OGG,DFW,1855,1855.0,0.0,16.0,1911.0,...,425.0,467.0,440.0,3711.0,0.0,0.0,42.0,0.0,0.0,NaN
5,2017-01-01,AA,7,DFW,OGG,940,1619.0,399.0,12.0,1631.0,...,503.0,498.0,480.0,3711.0,394.0,0.0,0.0,0.0,0.0,NaN
6,2017-01-01,AA,8,HNL,DFW,1838,1903.0,25.0,19.0,1922.0,...,432.0,465.0,434.0,3784.0,25.0,0.0,33.0,0.0,0.0,NaN
7,2017-01-01,AA,9,JFK,SFO,700,656.0,-4.0,22.0,718.0,...,405.0,387.0,362.0,2586.0,NaN,NaN,NaN,NaN,NaN,NaN
8,2017-01-01,AA,10,LAX,JFK,2100,2100.0,0.0,15.0,2115.0,...,327.0,297.0,272.0,2475.0,NaN,NaN,NaN,NaN,NaN,NaN
9,2017-01-01,AA,12,SFO,JFK,1135,1130.0,-5.0,27.0,1157.0,...,343.0,324.0,280.0,2586.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#drop columns that all have NA values
dataset.dropna(axis='columns',how="all",inplace=True)


In [6]:
display(dataset.head(n=10))

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2017-01-01,AA,1,JFK,LAX,800,831.0,31.0,25.0,856.0,...,0.0,402.0,398.0,347.0,2475.0,27.0,0.0,0.0,0.0,0.0
1,2017-01-01,AA,2,LAX,JFK,900,934.0,34.0,34.0,1008.0,...,0.0,327.0,335.0,289.0,2475.0,34.0,0.0,8.0,0.0,0.0
2,2017-01-01,AA,4,LAX,JFK,1130,1221.0,51.0,20.0,1241.0,...,0.0,328.0,319.0,284.0,2475.0,7.0,0.0,0.0,0.0,35.0
3,2017-01-01,AA,5,DFW,HNL,1135,1252.0,77.0,19.0,1311.0,...,0.0,517.0,537.0,513.0,3784.0,77.0,0.0,20.0,0.0,0.0
4,2017-01-01,AA,6,OGG,DFW,1855,1855.0,0.0,16.0,1911.0,...,0.0,425.0,467.0,440.0,3711.0,0.0,0.0,42.0,0.0,0.0
5,2017-01-01,AA,7,DFW,OGG,940,1619.0,399.0,12.0,1631.0,...,0.0,503.0,498.0,480.0,3711.0,394.0,0.0,0.0,0.0,0.0
6,2017-01-01,AA,8,HNL,DFW,1838,1903.0,25.0,19.0,1922.0,...,0.0,432.0,465.0,434.0,3784.0,25.0,0.0,33.0,0.0,0.0
7,2017-01-01,AA,9,JFK,SFO,700,656.0,-4.0,22.0,718.0,...,0.0,405.0,387.0,362.0,2586.0,NaN,NaN,NaN,NaN,NaN
8,2017-01-01,AA,10,LAX,JFK,2100,2100.0,0.0,15.0,2115.0,...,0.0,327.0,297.0,272.0,2475.0,NaN,NaN,NaN,NaN,NaN
9,2017-01-01,AA,12,SFO,JFK,1135,1130.0,-5.0,27.0,1157.0,...,0.0,343.0,324.0,280.0,2586.0,NaN,NaN,NaN,NaN,NaN


In [13]:
#print columns of dataset and their datatype
print(dataset.dtypes)

FL_DATE                 object
OP_CARRIER              object
OP_CARRIER_FL_NUM        int64
ORIGIN                  object
DEST                    object
CRS_DEP_TIME             int64
DEP_TIME               float64
DEP_DELAY              float64
TAXI_OUT               float64
WHEELS_OFF             float64
WHEELS_ON              float64
TAXI_IN                float64
CRS_ARR_TIME             int64
ARR_TIME               float64
ARR_DELAY              float64
CANCELLED              float64
CANCELLATION_CODE       object
DIVERTED               float64
CRS_ELAPSED_TIME       float64
ACTUAL_ELAPSED_TIME    float64
AIR_TIME               float64
DISTANCE               float64
CARRIER_DELAY          float64
WEATHER_DELAY          float64
NAS_DELAY              float64
SECURITY_DELAY         float64
LATE_AIRCRAFT_DELAY    float64
dtype: object


In [23]:
#print percantage of NA values for each feature
for col in dataset.columns:
   print(col,(sum(pd.isna(dataset[col])/dataset.shape[0])*100))

FL_DATE 0.0
OP_CARRIER 0.0
OP_CARRIER_FL_NUM 0.0
ORIGIN 0.0
DEST 0.0
CRS_DEP_TIME 0.0
DEP_TIME 1.4945996168338376
DEP_DELAY 1.5330227566262846
TAXI_OUT 1.5361108845821396
WHEELS_OFF 1.5360720890048047
WHEELS_ON 1.5822388260337314
TAXI_IN 1.5822388260337314
CRS_ARR_TIME 0.0
ARR_TIME 1.5822310669182642
ARR_DELAY 1.8020623263334392
CANCELLED 0.0
CANCELLATION_CODE 98.45378677874118
DIVERTED 0.0
CRS_ELAPSED_TIME 0.0001319049629397489
ACTUAL_ELAPSED_TIME 1.7819041443497945
AIR_TIME 1.7819041443497945
DISTANCE 0.0
CARRIER_DELAY 81.51636705945558
WEATHER_DELAY 81.51636705945558
NAS_DELAY 81.51636705945558
SECURITY_DELAY 81.51636705945558
LATE_AIRCRAFT_DELAY 81.51636705945558


In [24]:
# drop cancellation code - 98% missing and irrelevant
dataset.drop('CANCELLATION_CODE', inplace=True, axis=1)

In [31]:
dataset.shape

(12888067, 26)

In [28]:
dataset1 = dataset.dropna()

In [29]:
display(dataset1.head(n=10))

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2017-01-01,AA,1,JFK,LAX,800,831.0,31.0,25.0,856.0,...,0.0,402.0,398.0,347.0,2475.0,27.0,0.0,0.0,0.0,0.0
1,2017-01-01,AA,2,LAX,JFK,900,934.0,34.0,34.0,1008.0,...,0.0,327.0,335.0,289.0,2475.0,34.0,0.0,8.0,0.0,0.0
2,2017-01-01,AA,4,LAX,JFK,1130,1221.0,51.0,20.0,1241.0,...,0.0,328.0,319.0,284.0,2475.0,7.0,0.0,0.0,0.0,35.0
3,2017-01-01,AA,5,DFW,HNL,1135,1252.0,77.0,19.0,1311.0,...,0.0,517.0,537.0,513.0,3784.0,77.0,0.0,20.0,0.0,0.0
4,2017-01-01,AA,6,OGG,DFW,1855,1855.0,0.0,16.0,1911.0,...,0.0,425.0,467.0,440.0,3711.0,0.0,0.0,42.0,0.0,0.0
5,2017-01-01,AA,7,DFW,OGG,940,1619.0,399.0,12.0,1631.0,...,0.0,503.0,498.0,480.0,3711.0,394.0,0.0,0.0,0.0,0.0
6,2017-01-01,AA,8,HNL,DFW,1838,1903.0,25.0,19.0,1922.0,...,0.0,432.0,465.0,434.0,3784.0,25.0,0.0,33.0,0.0,0.0
15,2017-01-01,AA,21,JFK,LAX,1900,1922.0,22.0,20.0,1942.0,...,0.0,407.0,416.0,359.0,2475.0,22.0,0.0,9.0,0.0,0.0
19,2017-01-01,AA,28,LAX,JFK,2200,2202.0,2.0,22.0,2224.0,...,0.0,327.0,350.0,273.0,2475.0,2.0,0.0,23.0,0.0,0.0
24,2017-01-01,AA,33,JFK,LAX,700,656.0,-4.0,26.0,722.0,...,0.0,400.0,446.0,363.0,2475.0,0.0,0.0,42.0,0.0,0.0


In [30]:
dataset1.shape

(2381848, 26)

In [41]:
# wait for data
dataset['CANCELLED'].unique()

array([0., 1.])

In [39]:
# wait for data
df_predic = dataset1[dataset1['CANCELLED'] == 1.0 ]

In [40]:
# wait for data
display(df_predic.head(n=10))
df_predic.shape

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY


(0, 26)